In [8]:
import pandas as pd
import numpy as np
import os
import spacy
from os.path import exists
from spacy.tokens import DocBin
import random


Experimentation with a Single File

In [14]:
df = pd.read_csv(r"C:\Users\Pruthvi\Desktop\trainingannotations\2014-20335_Case_Report_-_Redacted00001-10.jpg.txt.csv")
df.iloc[:,1:]

,Line,Begin Offset,End Offset,Type
0,3,0,13,CASE NUMBER
1,9,90,97,OFFICER
2,10,81,88,OFFICER
3,12,22,29,OFFICER
4,13,56,63,OFFICER
5,14,55,62,OFFICER
6,16,91,98,OFFICER
7,19,19,26,OFFICER
8,22,75,89,SUBJECT
9,23,53,60,OFFICER


In [15]:


#open file
file = open(r"C:\Users\Pruthvi\Desktop\output\2014-20335_Case_Report_-_Redacted00001-01.jpg.txt")
training_data = list([])

#get dataframe
df = pd.read_csv(r"C:\Users\Pruthvi\Desktop\trainingannotations\2014-20335_Case_Report_-_Redacted00001-01.jpg.txt.csv")

#get relevant line numbers and words' offsets
allLines = df['Line']
bo = df['Begin Offset']
eo = df['End Offset']
labels = df['Type']
counter = 0

#loop through the file's lines and get the words
for LIndex, L in enumerate(file.readlines()):
    if LIndex  == allLines[counter]:
        temp = tuple((L, [(bo[counter],eo[counter],labels[counter])]))
        training_data.append(temp)
        counter+=1
    
    

The Actual Code

In [16]:
training_data = list([])

directory = r"C:\Users\Pruthvi\Desktop\output"
for file_name in os.listdir(directory):
    if exists(r"C:\Users\Pruthvi\Desktop\trainingannotations\\" + file_name + ".csv"):
        #print("New File")
        
        #open file
        file = open(r"C:\Users\Pruthvi\Desktop\output\\" + file_name)
        #print(file_name)
        
        #get dataframe
        df = pd.read_csv(r"C:\Users\Pruthvi\Desktop\trainingannotations\\" + file_name + ".csv").sort_values(by="Line", ascending=True)
        df_length = len(df)
        df = df.sort_values(by="Line", ascending=True)
        df.index = range(df_length)
        #print(df_length)
        #print("")

        #if the dataframe is empty
        if df_length ==0:
            continue
        
        
        #get relevant line numbers and words' offsets
        allLines = df['Line']
        bo = df['Begin Offset']
        eo = df['End Offset']
        labels = df['Type']
        counter = 0

        #loop through the file's lines and get the words
        for LIndex, L in enumerate(file.readlines()):
            #print("Counter:" + str(counter))
            #print("Index:" + str(LIndex))
            #print("Next Line Number:" + str(allLines[counter]))
            if LIndex == allLines[counter]:
                #print('Index in')
                temp = tuple((L, [(bo[counter],eo[counter],labels[counter])]))
                training_data.append(temp)
                #print(training_data[len(training_data)-1])
                
                if counter+1 < df_length:
                    if allLines[counter+1] > allLines[counter]:
                        counter+=1
                    
            
            

In [17]:
#Randomize the data before we split it for training and validation
random.shuffle(training_data)

#Calculate split point
split_position = int(len(training_data)*0.7)

#Create training and validation sets
validation = training_data[split_position:]
training = training_data[:split_position]

In [18]:
nlp = spacy.blank("en")

#Saving Training data to .spacy format file
db = DocBin()
for text, annotations in training:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span != None:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./train.spacy")


# Saving Validation data to .spacy format file
db = DocBin()
for text, annotations in validation:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span != None:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./valid.spacy")


In [19]:
#Create a base config file on spacy website and download

#Add paths to training and validation sets in base config file and convert to regular config file
#python -m spacy init fill-config base_config.cfg config.cfg

#Train the model
#python -m spacy train config.cfg --output ./ner_model


In [20]:
ner = spacy.load("ner_model/model-best")

In [24]:
#Get predictions for Santa Maria Data
SantaMaria = r"C:\Users\Pruthvi\Desktop\policingnotebook\Training Data\Santa Maria PD\txt"
SantaMariaPredictions = []
for file_name in os.listdir(SantaMaria):
    file = open(SantaMaria + "\\" + file_name)
    try: 
        textarr = file.readlines()
        textstring = ner(" ".join(textarr))

        predictions = ["FILE NAME: " + file_name]
        for ent in textstring.ents:
            
            predictions += [[ent.text, ent.label_]]

        SantaMariaPredictions += [predictions]
    
    except(UnicodeDecodeError):
        continue

#Save predictions to txt file
SM_txt = open(r"C:\Users\Pruthvi\Desktop\policingnotebook\Predictions\SantaMariaPredictions.txt", "w")
counter = 1
SantaMariaString = ""

for doc in SantaMariaPredictions:
    SantaMariaString += "File: " + str(counter) + "\n"
    
    if len(doc) > 1:
        for pred in doc[1:]:
            SantaMariaString += pred[0] + ": " + pred[1] + '\n'

    SantaMariaString += "\n"
    counter+=1
SM_txt.write(SantaMariaString)


204805

In [25]:
#Get predictions for Nevada County Data
NevadaCounty = r"C:\Users\Pruthvi\Desktop\policingnotebook\Training Data\Nevada County\txt"
NevadaCountyPredictions = []
for file_name in os.listdir(NevadaCounty):
    file = open(NevadaCounty + "\\" + file_name)
    try: 
        textarr = file.readlines()
        textstring = ner(" ".join(textarr))

        predictions = ["FILE NAME: " + file_name]
        for ent in textstring.ents:
            
            predictions += [[ent.text, ent.label_]]

        NevadaCountyPredictions += [predictions]
    
    except(UnicodeDecodeError):
        continue

#Save predictions to txt file
NC_txt = open(r"C:\Users\Pruthvi\Desktop\policingnotebook\Predictions\NevadaCountyPredictions.txt", "w")
counter = 1
NevadaCountyString = ""

for doc in NevadaCountyPredictions:
    NevadaCountyString += "File: " + str(counter) + "\n"
    
    if len(doc) > 1:
        for pred in doc[1:]:
            NevadaCountyString += pred[0] + ": " + pred[1] + '\n'

    NevadaCountyString += "\n"
    counter+=1
NC_txt.write(NevadaCountyString)

62614

In [27]:
#Get predictions for Alameda County Data
AlamedaCounty = r"C:\Users\Pruthvi\Desktop\policingnotebook\Training Data\AlamedaCountyFiles\txt"
AlamedaCountyPredictions = []
for file_name in os.listdir(AlamedaCounty):
    file = open(AlamedaCounty + "\\" + file_name)
    try: 
        textarr = file.readlines()
        textstring = ner(" ".join(textarr))

        predictions = ["FILE NAME: " + file_name]
        for ent in textstring.ents:
            
            predictions += [[ent.text, ent.label_]]

        AlamedaCountyPredictions += [predictions]
    
    except(UnicodeDecodeError):
        continue

#Save predictions to txt file
AC_txt = open(r"C:\Users\Pruthvi\Desktop\policingnotebook\Predictions\AlamedaCountyPredictions.txt", "w")
counter = 1
AlamedaCountyString = ""

for doc in AlamedaCountyPredictions:
    AlamedaCountyString += "File: " + str(counter) + "\n"
    
    if len(doc) > 1:
        for pred in doc[1:]:
            AlamedaCountyString += pred[0] + ": " + pred[1] + '\n'

    AlamedaCountyString += "\n"
    counter+=1
AC_txt.write(AlamedaCountyString)

47967